# Topic 6: Agent-Based Modeling (ABM)  
### Simulating Ideological Drift Between AI-Attachment and Grievance Communities

## 1. Project Context & Motivation

This notebook represents the final stage of the broader *Radicalisation Drift* project, in which we studied whether participation in AI-Companion subreddits may, for a small subset of users, function as an entry point into online grievance ecosystems.  

Previous notebooks provided strong evidence that:

- **Sentiment** in Grievance communities shows a long-term negative drift, while AI communities remain comparatively positive.  
- **Topic structure** and **linguistic features** differ sharply between the two ecosystems (NLP notebooks).  
- **Social network analysis (SNA)** revealed partially separated, modular clusters with a few “bridge” subreddits through which user migration occurs.

However, all analytical methods up to this point remain *observational*.  
To understand whether the structure of user interactions **could theoretically generate ideological drift**, we turn to **Agent-Based Modeling**.


## Research Question

**How does the combination of emotional tone and social influence shape ideological stability, and under what conditions can a small “toxic core” drive the system toward grievance-oriented radicalization?**

This question is grounded in:

- **Homophily theory** (Lazarsfeld & Merton, 1954) → people adjust beliefs toward similar peers  
- **Opinion dynamics** (DeGroot 1974; Friedkin & Johnsen 1999) → social averaging generates drift  
- **Online radicalization cascade models** (Baumann et al. 2020; Ribeiro et al. 2021) → small extremist cores can shift group norms over time  

ABM allows us to test whether these mechanisms *could* reproduce the patterns observed in real Reddit data.


## Why ABM?

Agent-Based Models are uniquely suited because:

- they simulate **micro-level interactions** (user-to-user influence)  
- which produce **macro-level outcomes** (population-wide ideological shifts)  
- capturing tipping points, cascades, stability, and influence of extreme minorities  

This notebook therefore functions both as:
1) a conceptual extension of the empirical analysis  
2) a robustness test of whether the observed drift is *structurally plausible*


## 2. Model Specification: Opinion Dynamics on a Network

| **Element**                | **Definition**                        | **Rationale**                                          |
| -------------------------- | ------------------------------------- | ------------------------------------------------------ |
| Agents                | 1000 virtual users                    | Scales to Reddit-like meso-communities                 |
| State (Oᵢ)             | Continuous opinion in ([-1, 1])       | -1 = grievance/radical, +1 = AI-attachment             |
| Initial Opinions       | Normal distribution (μ = 0.1)         | Reflects empirical positivity of AI communities        |
| Network Structure      | Small-World (Watts–Strogatz)          | Captures clustered communities with short paths        |
| Core Mechanism         | Influence = mean opinion of neighbors | Based on DeGroot (1974) opinion averaging              |
| Susceptibility α       | Strength of social influence          | Parameter varied in sensitivity analysis               |
| Shock Condition        | “Toxic core” fixed at –0.99           | Models extremist minorities from radicalization theory |
| Entrenchment Threshold | ( Oᵢ > 0.95 )                        | Represents irreversible ideological commitment         |


This model simulates whether local influence can cause large-scale ideological drift.


In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import random
import seaborn as sns
import plotly.express as px


In [71]:
# Core ABM Parameters
N_AGENTS = 1000
N_STEPS = 50
N_NEIGHBORS = 5
SUSCEPTIBILITY_ALPHA = 0.15   # Key parameter for sensitivity analysis


## 3. Model Implementation

We use a simple yet well-established opinion dynamics mechanism:

$$
O_i(t+1) = O_i(t) + \alpha \cdot \left( \overline{O}_{neighbors(i)} - O_i(t) \right)
$$

This corresponds to classical DeGroot/Friedkin-Johnsen models where:

- agents partially internalize the average opinion around them  
- influencers (the “toxic core”) remain fixed, exerting constant directional pressure  


In [72]:
# Opinion Drift Model with Optional Fixed Influencers 

class OpinionDriftModel:
    def __init__(self, N, N_steps, alpha, N_neighbors, n_influencers=0):
        """
        N              = number of agents
        N_steps        = total simulation steps
        alpha          = susceptibility to social influence
        N_neighbors    = average node degree in the small-world network
        n_influencers  = number of agents fixed at extreme negative opinion
        """
        
        self.N = N
        self.alpha = alpha
        self.schedule = range(N_steps)

        # Initial opinions (empirically informed: slightly positive bias)
        self.opinions = np.random.normal(loc=0.1, scale=0.3, size=N)

        # Toxic core (fixed negative influencers)
        self.influencer_mask = np.zeros(N, dtype=bool)
        if n_influencers > 0:
            influencer_indices = np.random.choice(N, n_influencers, replace=False)
            self.influencer_mask[influencer_indices] = True
            self.opinions[influencer_indices] = -0.99
            print(f"NOTE: {n_influencers} toxic core agents initialized. ")

        # Small-world network of interactions
        self.G = nx.watts_strogatz_graph(N, N_neighbors, p=0.1)

        self.history = []

    def step(self):
        """Simulates one time step of opinion updating."""
        new_opinions = self.opinions.copy()

        for i in range(self.N):

            # Influencers remain fixed
            if self.influencer_mask[i]:
                new_opinions[i] = self.opinions[i]
                continue

            # Get neighbors
            neighbors = list(self.G.neighbors(i))
            if not neighbors:
                continue

            # Influence = mean opinion of neighbors
            influence = np.mean(self.opinions[neighbors])

            # Social drift rule
            shift = self.alpha * (influence - self.opinions[i])
            new_opinions[i] = np.clip(self.opinions[i] + shift, -1, 1)

        self.opinions = new_opinions
        self.history.append(self.opinions.copy())

    def run_simulation(self):
        """Runs the full simulation and returns aggregated statistics."""
        for _ in self.schedule:
            self.step()

        self.df_history = pd.DataFrame(self.history)
        self.df_history['Step'] = self.df_history.index

        # Aggregate statistics per timestep
        self.df_stats = (
            self.df_history
            .melt(id_vars=['Step'], var_name='Agent')
            .groupby('Step')['value']
            .agg(
                mean_opinion='mean',
                radical_count=lambda x: (x < -0.95).sum(),
                ai_count=lambda x: (x > 0.95).sum()
            )
            .reset_index()
        )

        return self.df_stats


## 4. Sensitivity Analysis: Toxic Core vs Baseline

To test model robustness, we compare two scenarios:

1. **Baseline (No Influencers):**  
   Agents update opinions only through peer averaging.

2. **Shock Scenario (2% Toxic Core):**  
   20 agents are locked at a highly negative opinion (-0.99), representing an extremist minority.

Small extremist minorities are known in radicalization theory to disproportionately shift group norms (“minority influence”; Moscovici 1980).  
This ABM tests whether such an effect could arise under our empirically-informed parameters.


In [73]:
# Experimental Parameters
N_AGENTS_TOTAL = 1000
N_STEPS_TOTAL = 50
ALPHA_BASE = 0.15

# Scenario 1: Baseline 
model_base = OpinionDriftModel(
    N=N_AGENTS_TOTAL,
    N_steps=N_STEPS_TOTAL,
    alpha=ALPHA_BASE,
    N_neighbors=5,
    n_influencers=0
)
results_base = model_base.run_simulation()
results_base['Scenario'] = "Baseline (No Core)"

# Scenario 2: Toxic Core (2%) 
INFLUENCER_COUNT = 20  # 2% of the population
model_shock = OpinionDriftModel(
    N=N_AGENTS_TOTAL,
    N_steps=N_STEPS_TOTAL,
    alpha=ALPHA_BASE,
    N_neighbors=5,
    n_influencers=INFLUENCER_COUNT
)
results_shock = model_shock.run_simulation()
results_shock['Scenario'] = f"Shock (2% Toxic Core = {INFLUENCER_COUNT} Agents)"

# Combine data
df_comparison = pd.concat([results_base, results_shock], ignore_index=True)

print("Rendering interactive sensitivity plot...")


NOTE: 20 toxic core agents initialized. 
Rendering interactive sensitivity plot...


In [74]:
fig = px.line(
    df_comparison,
    x='Step',
    y='mean_opinion',
    color='Scenario',
    title='ABM Sensitivity: Impact of 2% Toxic Core on Ideological Drift',
    labels={'mean_opinion': 'Mean Opinion Score [-1, 1]', 'Step': 'Time Step'},
    height=600
)

fig.add_hline(y=0, line_dash="dash", line_color="grey", opacity=0.6, annotation_text="Neutral Baseline")

fig.update_layout(
    title_x=0.5,
    legend_title='Simulation Scenario',
    plot_bgcolor='white'
)

fig.update_traces(mode='lines+markers', marker_size=4)

fig.show()


## 5. Interpretation of Dynamics

The simulation shows a striking divergence:

### **Baseline**
The system remains stable around its initial slightly positive sentiment.  
This means that ordinary social influence **does not spontaneously produce radicalization** under the empirical parameters extracted from Reddit.

### **Toxic Core (2%)**
Even though only 20 agents are extremists:

- their fixed position at -0.99 exerts **constant downward pressure**  
- the mean opinion drifts steadily downward  
- by step ~45, the average opinion crosses into negative territory  

This reflects classical findings in computational sociology:

- **Minority influence theory** (Moscovici, 1980)  
- **Opinion drift cascades** (Friedkin & Johnsen 1999)  
- **Norm-shifting under persistent extremists** (Bail 2021; Ribeiro et al. 2021)

### **Implication**

This pattern aligns with classic findings from influence-based opinion models: even a very small but **stubborn** extremist subgroup can steadily pull the entire network away from its original state.  
In the context of the project, this illustrates how a small radical cluster within online communities can exert disproportionate influence on broader user ecosystems, making large-scale ideological drift structurally plausible.



## 6. Final Simulation Experiment: Baseline Stability + Shock Scenario

To evaluate both stochastic stability and the impact of a toxic minority, we run three simulations:

1. **Baseline Run 1** - no influencers  
2. **Baseline Run 2** - identical parameters, new random seed  
3. **Shock Scenario** - 2% toxic core fixed at −0.99  

All three runs are visualized together to show how baseline dynamics compare to a radicalizing external shock.


In [75]:
# Final Simulations: two baselines + shock scenario 

# Baseline Run 1: default dynamics with no fixed influencers.
model_base1 = OpinionDriftModel(
    N=N_AGENTS_TOTAL,
    N_steps=N_STEPS_TOTAL,
    alpha=ALPHA_BASE,
    N_neighbors=5,
    n_influencers=0
)
results_base1 = model_base1.run_simulation()
results_base1["Scenario"] = "Baseline Run 1"


# Baseline Run 2: same parameters, new random initialization.
# Used to check whether stochasticity changes the trajectory.
model_base2 = OpinionDriftModel(
    N=N_AGENTS_TOTAL,
    N_steps=N_STEPS_TOTAL,
    alpha=ALPHA_BASE,
    N_neighbors=5,
    n_influencers=0
)
results_base2 = model_base2.run_simulation()
results_base2["Scenario"] = "Baseline Run 2"


# Shock Scenario: introduces a small fixed “toxic core” (2%).
# These agents stay at −0.99 and exert constant negative pressure.
model_shock = OpinionDriftModel(
    N=N_AGENTS_TOTAL,
    N_steps=N_STEPS_TOTAL,
    alpha=ALPHA_BASE,
    N_neighbors=5,
    n_influencers=INFLUENCER_COUNT
)
results_shock = model_shock.run_simulation()
results_shock["Scenario"] = f"Shock (2% Toxic Core = {INFLUENCER_COUNT})"


# Combine all runs into one dataframe for plotting.
df_final = pd.concat([results_base1, results_base2, results_shock], ignore_index=True)


#  Visualization: compare opinion trajectories across scenarios 
fig = px.line(
    df_final,
    x='Step',
    y='mean_opinion',
    color='Scenario',
    title="Final ABM Comparison: Stable Baselines vs. Toxic-Core Shock",
    labels={'mean_opinion': 'Mean Opinion Score [-1, 1]', 'Step': 'Time Step'},
    height=600
)

# Add neutral reference line.
fig.add_hline(
    y=0,
    line_dash="dash",
    line_color="grey",
    opacity=0.6,
    annotation_text="Neutral Baseline"
)


fig.update_layout(
    title_x=0.5,
    legend_title="Simulation Scenario",
    plot_bgcolor="white"
)
fig.update_traces(mode="lines+markers", marker_size=4)

fig.show()


NOTE: 20 toxic core agents initialized. 


## Final Simulation Comparison: Baseline Stability vs. Toxic-Core Shock

This final experiment compares two independent baseline runs with a shock scenario in which 2% of agents are fixed at an extreme negative opinion (−0.99). The two baselines act as a robustness check: because the model includes stochastic initialization, separate runs ensure that system behaviour is not driven by randomness.

### Key Observations
- **Both baselines remain stable:** despite random variation in initial opinions and network structure, the mean opinion stays near the initial positive bias (~0.10).  
  This indicates that under normal conditions the population does *not* drift toward radicalization.

- **The shock scenario diverges immediately:** introducing a small, persistent extremist minority produces a clear, monotonic decline in the population’s average opinion.  
  The shift is gradual but systematic, reflecting continuous negative pressure from the fixed agents.

- **Cross-run consistency:** the baseline trajectories nearly overlap, strengthening confidence that the sharp downward drift in the shock scenario is caused by the toxic core rather than stochastic noise.


### Interpretation

These results fit the patterns we see in the Reddit data, where grievance-oriented/incel spaces are not inherently “extremist” by default, but they do maintain a consistently more negative emotional tone and attract large volumes of activity. Most users in these spaces are not radical; they are often frustrated or socially isolated rather than ideologically extreme.

What the simulation shows is that **the arrival or persistence of a small radical subgroup can sharply alter the overall trajectory**.  
The baseline system remains stable, but once even a tiny but popular fixed cluster with extreme negativity appears, the surrounding population begins to drift downward as well.

This does *not* predict actual behaviour, but it highlights a structural fragility typical for communities dealing with grievance-based topics:  
**without strong moderation and boundary-setting, the combination of high activity + negative tone makes these spaces unusually sensitive to radical actors.**

In other words, the problem is not the average user -  
it is how quickly the emotional climate can shift once a minority of committed radicals enters the environment.
